In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

# -----------------------------
# Configuration
# -----------------------------
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32
EPOCHS = 25
SEED = 42

TRAIN_DIR = r"D:\LeafUpdate_Split\train"
VAL_DIR = r"D:\LeafUpdate_Split\val"
TEST_DIR = r"D:\LeafUpdate_Split\test"

# -----------------------------
# Load Datasets
# -----------------------------
train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=SEED
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    VAL_DIR,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    shuffle=False,
    seed=SEED
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    shuffle=False,
    seed=SEED
)

# -----------------------------
# Normalize Images
# -----------------------------
def normalize(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_ds = train_ds.map(normalize)
val_ds = val_ds.map(normalize)
test_ds = test_ds.map(normalize)

# Prefetch for performance
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.prefetch(tf.data.AUTOTUNE)

# -----------------------------
# Custom CNN Architecture
# -----------------------------
def build_custom_cnn(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.25),

        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.25),

        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.3),

        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling2D(),

        layers.Dense(128, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(2, activation='softmax')  # 2 classes
    ])
    return model

model = build_custom_cnn()
model.summary()

# -----------------------------
# Compile Model
# -----------------------------
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# -----------------------------
# Callbacks
# -----------------------------
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint("custom_cnn_best.keras", monitor='val_accuracy', save_best_only=True)
]

# -----------------------------
# Train Model
# -----------------------------
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks
)

# -----------------------------
# Evaluate on Test Set
# -----------------------------
test_loss, test_acc = model.evaluate(test_ds)
print(f"\n Custom CNN Test Accuracy: {test_acc*100:.2f}%")

# -----------------------------
# Save Model
# -----------------------------
model.save("custom_cnn_final_model.keras")
print("\n Model saved successfully as 'custom_cnn_final_model.keras'")


Found 8415 files belonging to 2 classes.
Found 1803 files belonging to 2 classes.
Found 1806 files belonging to 2 classes.


C:\Users\kavya achari\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 28, 28, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 423,490 (1.62 MB)

 Trainable params: 422,530 (1.61 MB)

 Non-trainable params: 960 (3.75 KB)

Epoch 1/25
263/263 ━━━━━━━━━━━━━━━━━━━━ 549s 2s/step - accuracy: 0.9313 - loss: 0.2042 - val_accuracy: 0.5169 - val_loss: 4.7197
Epoch 2/25
263/263 ━━━━━━━━━━━━━━━━━━━━ 482s 2s/step - accuracy: 0.9657 - loss: 0.0974 - val_accuracy: 0.5169 - val_loss: 3.7139
Epoch 3/25
263/263 ━━━━━━━━━━━━━━━━━━━━ 485s 2s/step - accuracy: 0.9743 - loss: 0.0745 - val_accuracy: 0.7976 - val_loss: 0.4023
Epoch 4/25
263/263 ━━━━━━━━━━━━━━━━━━━━ 485s 2s/step - accuracy: 0.9764 - loss: 0.0640 - val_accuracy: 0.9540 - val_loss: 0.1649
Epoch 5/25
263/263 ━━━━━━━━━━━━━━━━━━━━ 485s 2s/step - accuracy: 0.9843 - loss: 0.0489 - val_accuracy: 0.9662 - val_loss: 0.1257
Epoch 6/25
263/263 ━━━━━━━━━━━━━━━━━━━━ 484s 2s/step - accuracy: 0.9836 - loss: 0.0454 - val_accuracy: 0.9606 - val_loss: 0.1586
Epoch 7/25
263/263 ━━━━━━━━━━━━━━━━━━━━ 1584s 6s/step - accuracy: 0.9865 - loss: 0.0391 - val_accuracy: 0.9662 - val_loss: 0.1407
Epoch 8/25
263/263 ━━━━━━━━━━━━━━━━━━━━ 734s 3s/step - accuracy: 0.9876 - loss: 0.0338 - val_acc

In [2]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Get true labels and predictions
y_true = np.concatenate([y for x, y in test_ds], axis=0)
y_pred_probs = model.predict(test_ds)
y_pred = np.argmax(y_pred_probs, axis=1)

# Classification Report
print("\n📊 Classification Report:")
print(classification_report(y_true, y_pred, target_names=['Healthy', 'Unhealthy']))

# Confusion Matrix
print("\n🧩 Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))


57/57 ━━━━━━━━━━━━━━━━━━━━ 16s 270ms/step

📊 Classification Report:
              precision    recall  f1-score   support

     Healthy       0.99      0.96      0.98       873
   Unhealthy       0.97      0.99      0.98       933

    accuracy                           0.98      1806
   macro avg       0.98      0.98      0.98      1806
weighted avg       0.98      0.98      0.98      1806


🧩 Confusion Matrix:
[[840  33]
 [  8 925]]


In [4]:
from tensorflow import keras


model = keras.models.load_model("custom_cnn_final_model.keras")


model.save("Cnn_model.h5")

print("✅ Model successfully saved as cnn_model.h5")


✅ Model successfully saved as cnn_model.h5


In [7]:
import tensorflow as tf

TEST_DIR = r"D:\LeafUpdate_Split\test"

test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    image_size=(224, 224),
    batch_size=32,
    shuffle=False
)

# Normalize
def normalize(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

test_ds = test_ds.map(normalize).prefetch(tf.data.AUTOTUNE)


Found 1806 files belonging to 2 classes.


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, confusion_matrix
import numpy as np

y_true = []
y_pred = []

for images, labels in test_ds:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

y_true = np.array(y_true)
y_pred = np.array(y_pred)

precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print(f"\n🔹 Overall Precision: {precision:.4f}")
print(f"🔹 Overall Recall: {recall:.4f}")
print(f"🔹 Overall F1-score: {f1:.4f}")




1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 498ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 533ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 477ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 334ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 389ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 398ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 392ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 361ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 381ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 366ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 388ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 405ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 370ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 389ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 387ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 405ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 389ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 395ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 384ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 379ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 416ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

AttributeError: '_PrefetchDataset' object has no attribute 'class_names'

In [1]:
from tensorflow import keras


model = keras.models.load_model("custom_cnn_final_model.keras")


model.save("cnn_model.h5")

print("✅ Model successfully saved as cnn_model.h5")


✅ Model successfully saved as cnn_model.h5


In [4]:
from tensorflow import keras

model = keras.models.load_model("custom_cnn_final_model.keras")

# Explicitly save in legacy HDF5 format
model.save("CNN_model.h5", save_format="h5")

print("✅ Model saved as CNN_model.h5 (HDF5 format)")

✅ Model saved as CNN_model.h5 (HDF5 format)


In [5]:
import os

print("Current working directory:", os.getcwd())
print("File exists?", os.path.exists("CNN_model.h5"))
print("Full path:", os.path.abspath("CNN_model.h5"))
print("File size:", os.path.getsize("CNN_model.h5") if os.path.exists("CNN_model.h5") else "Not found")

Current working directory: d:\Merge
File exists? True
Full path: d:\Merge\CNN_model.h5
File size: 5157088


In [6]:
from tensorflow import keras

model = keras.models.load_model("custom_cnn_final_model.keras")

# Explicitly save in legacy HDF5 format
model.save("CNN_Model.h5", save_format="h5")

print("✅ Model saved as CNN_Model.h5 (HDF5 format)")

✅ Model saved as CNN_Model.h5 (HDF5 format)


In [13]:
from tensorflow import keras

model = keras.models.load_model("custom_cnn_final_model.keras")

# Explicitly save in legacy HDF5 format
model.save("CNN_model_fixed.h5", save_format="h5")

print("✅ Model saved as CNN_model.h5 (HDF5 format)")

✅ Model saved as CNN_model.h5 (HDF5 format)


In [4]:
import tensorflow as tf
import keras

print("TF:", tf.__version__)
print("Keras:", keras.__version__)


TF: 2.20.0
Keras: 3.11.2


In [2]:
import tensorflow as tf

model = tf.keras.models.load_model("CNN_model_fixed.h5")

model.save("CNN_model_render.keras")


In [1]:
import tensorflow as tf

model = tf.keras.models.load_model("CNN_model_render.keras")
print("Model loaded successfully")
model.summary()


Model loaded successfully


C:\Users\kavya achari\AppData\Roaming\Python\Python310\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 42 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 28, 28, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,268,552 (4.84 MB)

 Trainable params: 422,530 (1.61 MB)

 Non-trainable params: 960 (3.75 KB)

 Optimizer params: 845,062 (3.22 MB)

In [2]:
from tensorflow.keras.models import load_model

model = load_model("CNN_model_render.keras")
print("Model loaded successfully")


Model loaded successfully


C:\Users\kavya achari\AppData\Roaming\Python\Python310\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 42 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [3]:
from tensorflow.keras.models import load_model

model = load_model("CNN_model_render.keras")
print("SUCCESS")


SUCCESS
